In [2]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle as pkl

from glob import glob

from dlab.nwbtools import df_from_phy, load_phy_template, option234_positions, load_unit_data, make_spike_secs

import pynwb
from hdmf.backends.hdf5.h5_utils import H5DataIO
from hdmf.utils import StrDataset

from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile

%matplotlib inline
%load_ext autoreload
%autoreload 2

warnings.simplefilter(action='ignore', category=UserWarning)

no Allen Institute NWB API. get this from http://stash.corp.alleninstitute.org/projects/INF/repos/ainwb/browse
no OpenEphys.py get this from https://github.com/open-ephys/analysis-tools


ImportError: cannot import name 'make_spike_secs' from 'dlab.nwbtools' (C:\Users\denmanlab\AppData\Roaming\Python\Python311\site-packages\dlab\nwbtools.py)

## Gather Spike and Stimulus Data

In [1]:
recording_path =  r'G:/Grant/rotation/2023-09-20_14-25-05_eOPN3_33/'
data_path = os.path.abspath(os.path.join(recording_path,*[os.pardir]*4))
old_dir = os.getcwd()
os.chdir(data_path)


experimenter    = 'grh'
experiment_name = 'eOPN3_33_20230920'
experiment_desc = '3 probe recording of eOPN3_33 using roation project stimuli. black white flashes (100), white noise matrix, drifting grating, naturalistic images'
lab             = 'Daniel Denman Lab'
institution     = 'University of Colorado, Anschutz Medical Campus'
session_id      = os.path.basename(os.path.abspath(os.path.join(recording_path,*[os.pardir]*3)))

NameError: name 'os' is not defined

In [3]:
stim_data = pd.read_json(glob('intermediates/*stim*')[0])

stim_data.head()

,stim,start,green,uv,sweep,timestamp,ori,x,y
0,Binary Luminance Noise,204.254167,"[[1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, -1, -1...","[[1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, -1, -1...",NaN,NaN,NaN,NaN,NaN
1,Binary Luminance Noise,204.278633,"[[-1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1,...","[[-1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1,...",NaN,NaN,NaN,NaN,NaN
2,Binary Luminance Noise,204.312100,"[[1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1...","[[1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1...",NaN,NaN,NaN,NaN,NaN
3,Binary Luminance Noise,204.345567,"[[1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1...","[[1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1...",NaN,NaN,NaN,NaN,NaN
4,Binary Luminance Noise,204.379033,"[[-1, 1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -...","[[-1, 1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -...",NaN,NaN,NaN,NaN,NaN


In [4]:
unit_data = pd.read_json(glob('intermediates/*unit*')[0])

unit_data.head()


,probe,cluster_id,group,depth,ch,xpos,ypos,template,weights,Amplitude,amplitudes,times,n_spikes,fr,ContamPct,KSLabel
0,A,0,good,400.0,0,37.0,1240.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",3326.4,"[33.0842056274, 32.8041152954, 31.2109260559, ...","[34.2384358989, 35.8248245337, 35.8295292252, ...",2506,0.628364,0.0,good
1,A,1,good,400.0,1,37.0,1240.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",1265.5,"[9.1411237717, 11.5981779099, 8.9386949539, 10...","[33.363594839, 33.5295602967, 33.6905229602, 3...",80943,20.295952,4.6,good
2,A,2,noise,400.0,1,37.0,1240.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",1656.9,"[14.7545623779, 18.8596878052, 18.714132309, 1...","[44.595181952, 67.6421773402, 129.7825191957, ...",144,0.036107,0.0,mua
3,A,3,mua,400.0,1,37.0,1240.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",1791.7,"[15.4172449112, 11.5067329407, 11.2728691101, ...","[64.9875844983, 69.3817072568, 71.7200426102, ...",765,0.191819,0.0,mua
4,A,4,good,400.0,2,37.0,1240.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",2167.3,"[24.0684566498, 20.7031936646, 19.0640583038, ...","[33.3580226172, 33.3650629694, 33.3956601396, ...",12937,3.243872,1.7,good


### Add HERBS Data

## Create NWB

In [49]:
nwbfile = NWBFile(identifier             = experiment_name, 
                  session_start_time     = datetime.now(tzlocal()),
                  experimenter           = experimenter,
                  lab                    = lab,
                  institution            = institution,
                  session_description = experiment_desc,
                  session_id             = session_id,
                  )

Add Epochs

In [50]:
epochs = np.unique(stim_data.stim)

for stimulus in epochs:
    nwbfile.add_epoch(stim_data[stim_data.stim == stimulus].start.values[0],
                      stim_data[stim_data.stim == stimulus].start.values[-1],
                      stimulus)

Add columns for describing each trial

In [51]:
stim_data.keys()

Index(['stim', 'start', 'green', 'uv', 'sweep', 'timestamp', 'ori', 'x', 'y'], dtype='object')

In [52]:
nwbfile.add_trial_column(name        = 'stimulus',  
                         description = 'stimulus name')
nwbfile.add_trial_column(name        = 'frame_no', 
                         description = 'index of presented frame')
nwbfile.add_trial_column(name        = 'green',    
                         description = 'green layer of presented frame')
nwbfile.add_trial_column(name        = 'uv',       
                         description = 'uv layer of presented frame')
nwbfile.add_trial_column(name        = 'sweep',    
                         description = 'index of sweep')
nwbfile.add_trial_column(name        = 'ori',     
                         description = 'sweeping bar orientation')
nwbfile.add_trial_column(name        = 'x',        
                         description = 'x position of bar center')
nwbfile.add_trial_column(name        = 'y',       
                         description = 'y position of bar center')


Fill trial columns with stimulus data

In [53]:
def nan2nan(cell):
    try:
        a = [np.nan if pd.isnull(x.all()) else x for x in [cell]][0]
    except:
        try:
            a = [np.nan if pd.isnull(x).all() else x for x in [cell]][0]
        except:
            a = [np.nan if pd.isnull(x) else x for x in [cell]][0]
    return a

In [54]:
for stimulus in epochs:
    print(stimulus)
    stim = stim_data[stim_data.stim == stimulus]
    stim.reset_index(drop=True,inplace=True)
    mean_diff = np.mean(np.diff(stim.start.values))
    for i, trial_start in enumerate(stim.start.values):

        nwbfile.add_trial(start_time = trial_start,
                          stop_time  = trial_start + mean_diff,
                          stimulus   = stimulus,
                          frame_no   = i,
                          green      = nan2nan(stim.green[i]),
                          uv         = nan2nan(stim.uv[i]),
                          sweep      = nan2nan(stim.sweep[i]),
                          ori        = nan2nan(stim.ori[i]),
                          x          = nan2nan(stim.x[i]),
                          y          = nan2nan(stim.y[i]),
                        )

Binary Luminance Noise
Gaussian Luminance Noise
Sweeping Bar
Ternary Phosphor Noise


In [55]:
probes = np.unique(unit_data.probe)
device = nwbfile.create_device('DenmanLab_EphysRig1','3 probe rig')

for i, probe in enumerate(probes):
    electrode_name = 'Probe '+str(i)
    description = "Neuropixels1.0_"+probes[i]
    location = "somewhere in the brain"

    electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                     description=description,
                                                     location=location,
                                                     device=device)
    
    #add channels to each probe
    for ch in range(option234_positions.shape[0]):
        nwbfile.add_electrode(x=option234_positions[ch,0],y=0.,z=option234_positions[0,1],imp=0.0,location='none',filtering='high pass 300Hz',group=electrode_group)
        
electrode_table_region = nwbfile.create_electrode_table_region([0], 'the second electrode')


In [56]:
nwbfile.add_unit_column(name        = 'probe', 
                        description = 'probe ID')

nwbfile.add_unit_column(name        = 'unit_id',
                        description = 'cluster ID from KS2')

nwbfile.add_unit_column(name        = 'group', 
                        description = 'user label of good/mua')

nwbfile.add_unit_column(name        = 'KSLabel', 
                        description = 'Kilosort label')

# nwbfile.add_unit_column(name        = 'qmLabel', 
#                         description = 'Bombcell label')

nwbfile.add_unit_column(name        = 'depth', 
                        description = 'the depth of this unit from zpos and insertion depth')

nwbfile.add_unit_column(name        = 'ch', 
                        description = 'channel number on probe')

nwbfile.add_unit_column(name        = 'xpos', 
                        description = 'the x position on probe')

nwbfile.add_unit_column(name        = 'ypos', 
                        description = 'the y position on probe')

# nwbfile.add_unit_column(name        = 'template',
#                         description = 'Template at each channel across timepoints [nTimepoints,nTempChannels]')

nwbfile.add_unit_column(name        = 'amplitudes',
                        description = 'Amplitude of each spike over time',index=True)

nwbfile.add_unit_column(name        = 'n_spikes', 
                        description = 'total number of spikes across recording')

nwbfile.add_unit_column(name        = 'fr', 
                        description = 'firing rate across recording')

nwbfile.add_unit_column(name        = 'amp', 
                        description = 'Kilosort amplitude')

nwbfile.add_unit_column(name        = 'ContamPct', 
                        description = 'Kilosort ISI contamination')

In [57]:
unit_data.keys()

Index(['probe', 'cluster_id', 'group', 'depth', 'ch', 'xpos', 'ypos',
       'template', 'weights', 'Amplitude', 'amplitudes', 'times', 'n_spikes',
       'fr', 'ContamPct', 'KSLabel'],
      dtype='object')

In [58]:
for i,unit_row in unit_data.iterrows():
    nwbfile.add_unit(probe       = str(unit_row.probe),
                     id          = int(unit_row.name), #built-in
                     unit_id     = unit_row.cluster_id,
                     spike_times = unit_row.times, #built-in
                     electrodes  = np.where(np.array(unit_row.weights) > 0)[0], #built-in
                     depth       = unit_row.depth,
                     ch          = unit_row.ch,
                     xpos        = unit_row.xpos,
                     ypos        = unit_row.ypos,
                    #  template    = np.array(unit_row.template,dtpye='object'),
                     amplitudes  = np.array(unit_row.amplitudes,dtype='object'),
                     n_spikes    = unit_row.n_spikes,
                     fr          = unit_row.fr,
                     group       = str(unit_row.group),
                     KSLabel     = str(unit_row.KSLabel),
                     amp         = unit_row.Amplitude,
                     ContamPct   = unit_row.ContamPct
                     )

In [59]:
nwb_path = os.path.join(data_path,os.path.basename(data_path))+'.nwb'
with pynwb.NWBHDF5IO(nwb_path, 'w') as io:
    io.write(nwbfile)

In [68]:
io = pynwb.NWBHDF5IO(nwb_path, mode='r')
nwb = io.read(skip_version_check=True)

UnsupportedOperation: Cannot read data from file C:\Users\juans\github\color_and_form\data\C140\C140.nwb in mode 'r'. There are no values.